In [4]:
import torch
import torch.nn as nn
from GPT_module import TransformerBlock, LayerNorm


In [5]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,  # vocabsize from BPE tokenizer
    "context_length": 1024,  # context length
    "emb_dim": 768,  # embedding dimension
    "n_heads": 12,  # number of attention heads
    "n_layers": 12,  # number of layers
    "drop_rate": 0.1,  # dropout rate
    "qkv_bias": False  # query-key-value bias
}


**GPT Model**

In [7]:


class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb=nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb=nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb=nn.Dropout(cfg["drop_rate"])

        self.trf_blocks=nn.Sequential(
            *[TransformerBlock(cfg=cfg) for _ in range(cfg["n_layers"])]
        )

        self.final_norm=LayerNorm(cfg["emb_dim"])
        self.out_head=nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )



    def forward(self, in_idx):
        batch_size, seq_len=in_idx.shape
        tok_embeds=self.tok_emb(in_idx)
        pos_embeds=self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x=tok_embeds+pos_embeds
        x=self.drop_emb(x)
        x=self.trf_blocks(x)
        x=self.final_norm(x)
        logits=self.out_head(x)
        return logits





In [9]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = 'Every effort moves you'
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)

print(batch)  # token IDs for 2 texts
print(batch.shape)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])
torch.Size([2, 4])


In [10]:
torch.manual_seed(123)
model=GPTModel(GPT_CONFIG_124M)

out=model(batch)
print(out.shape)

torch.Size([2, 4, 50257])


In [15]:
# total number of parameters
total_params=sum(p.numel() for p in model.parameters())
print(f"{total_params:,}")

163,009,536


In [12]:
# the memory requirment for 163 million parameters
total_size_bytes=total_params*4
total_size_mb=total_size_bytes/(1024*1024)
print(total_size_mb)

621.83203125


In [13]:
GPT_CONFIG_MEDIUM = {
    "vocab_size": 50257,  # vocabsize from BPE tokenizer
    "context_length": 1024,  # context length
    "emb_dim": 1024,  # embedding dimension
    "n_heads": 16,  # number of attention heads
    "n_layers": 24,  # number of layers for transformer block
    "drop_rate": 0.1,  # dropout rate
    "qkv_bias": False  # query-key-value bias
}


In [16]:
torch.manual_seed(123)
model_medium=GPTModel(GPT_CONFIG_MEDIUM)
total_params = sum(p.numel() for p in model_medium.parameters())
print(f"{total_params:,}")


406,212,608


In [17]:
GPT_CONFIG_LARGE = {
    "vocab_size": 50257,  # vocabsize from BPE tokenizer
    "context_length": 1024,  # context length
    "emb_dim": 1280,  # embedding dimension
    "n_heads": 20,  # number of attention heads
    "n_layers": 36,  # number of layers for transformer block
    "drop_rate": 0.1,  # dropout rate
    "qkv_bias": False  # query-key-value bias
}


In [18]:
torch.manual_seed(123)
model_large = GPTModel(GPT_CONFIG_LARGE)
total_params = sum(p.numel() for p in model_large.parameters())
print(f"{total_params:,}")


838,220,800


In [21]:
GPT_CONFIG_XL = {
    "vocab_size": 50257,  # vocabsize from BPE tokenizer
    "context_length": 1024,  # context length
    "emb_dim": 1600,  # embedding dimension
    "n_heads": 25,  # number of attention heads
    "n_layers": 48,  # number of layers for transformer block
    "drop_rate": 0.1,  # dropout rate
    "qkv_bias": False  # query-key-value bias
}


In [22]:
torch.manual_seed(123)
model_xl = GPTModel(GPT_CONFIG_XL)
total_params = sum(p.numel() for p in model_xl.parameters())
print(f"{total_params:,}")


1,637,792,000
